In [ ]:
import json
import re
from dataset import OpenMathDataset

def find_and_parse_json(s):
    # Attempt to find a JSON-like substring within the input string
    # This pattern is a simplified approach and might not cover all edge cases
    pattern = r"\{.*?\}"
    matches = re.findall(pattern, s, re.DOTALL)

    for match in matches:
        # Attempt to correct common deviations from the JSON standard:
        # Convert single quotes to double quotes (this might introduce errors if single quotes are used within strings)
        corrected_match = match.replace("'", '"')
        
        # Attempt to parse the corrected string as JSON
        try:
            parsed_json = json.loads(corrected_match)
            return parsed_json
        except json.JSONDecodeError:
            continue  # If parsing fails, continue to the next match
    
    # If no valid JSON object was found and successfully parsed, return None
    return None

not_json_mode_path = '/Users/arnaudstiegler/back-office-llm-bench/predictions_not_json_mode.json'
json_mode_path = '/Users/arnaudstiegler/back-office-llm-bench/predictions_json_mode.json'


preds = json.load(open(json_mode_path))
dataset = OpenMathDataset(json_mode=False)
contains_json = []
correct_json = []
correct_answer = []
for i in range(len(preds)):
    answer = preds[i]['choices'][0]['message']['content']
    sample = dataset[i+1]

    json_dict = find_and_parse_json(answer)

    if not json_dict:
        contains_json.append(0)
        print(answer)
        print('======')
    else:
        contains_json.append(1)

        if not set(json_dict.keys()) == {'answer'}:
            correct_json.append(0)
        else:
            correct_json.append(1)

            if str(json_dict['answer']) == str(sample.answer):
                correct_answer.append(1)
            else:
                correct_answer.append(0)

In [15]:
print(sum(contains_json) / len(contains_json))
print(sum(correct_json) / len(correct_json))
print(sum(correct_answer) / len(correct_answer))

0.35276532137518685
1.0
0.3474576271186441


In [3]:
# answer="{'effective_date': '2001-04-18', 'jurisdiction': 'Oregon', 'party': ['Eric_Dean_Sprunk', 'Nike_Inc.']}"

prompt = '''
The text above is the transcription of an NDA. 

# Extraction:
There are up to 6 attributes to be extracted from the transcription:

effective_date - date in YYYY-MM-DD format, at which point the contract is legally binding,
jurisdiction - under which state or country jurisdiction is the contract signed,
party - party or parties of the contract,
term - length of the legal contract as expressed in the document.
Note that party usually occur more than once.

# Normalization:
The expected pieces of information were normalized to some degree:

in attribute values, all spaces   and colons : were replaced with an underscores _,
all expected dates should be returned in YYYY-MM-DD format,
values for attribute term are normalized with the same original units e.g. eleven months is changed to 11_months; all of them are in the same format: {number}_{units}.

# Output Format:
The output has to be a valid json with the following format:
{"effective_date": "value", "jurisdiction":"value", "party":["value_1", "value_2", ...], "term":"value"}

So for instance:
"{'effective_date': '2020-01-12', 'jurisdiction': 'Utah', 'party': ['Bill_Gates', 'Coca_Cola_Inc.']}"

You can reason and explain your choices before returning the JSON object, you just have to have that json object in the output
'''

'71'

In [17]:
from dataset import KleisterNdaDataset

data = KleisterNdaDataset(json_mode=False)

In [20]:
data[1]

Sample(id=1, task_definition='', task_input='EX-99.(E)(2) 3 d450961dex99e2.htm EX-(E)(2)\\nExhibit (e)(2)\\nLOGO\\nConfidential\\nMUTUAL CONFIDENTIALITY AGREEMENT\\nThis Mutual Confidentiality Agreement (“Agreement”) is effective as of February 10, 2017 (“Effective Date”), between Kite Pharma,\\nInc., a Delaware corporation having a place of business at 2225 Colorado Avenue, Santa Monica, CA 90404 (“Kite”) and Gilead Sciences, Inc.,\\na Delaware corporation having a place of business at 333 Lakeside Drive, Foster City, CA 94404 (“Company”). Kite and Company may be\\nreferred to herein individually as a “Party” and collectively as the “Parties.”\\nThe Parties desire to exchange Confidential Information (as defined below) for the Purpose (as defined below) and desire that their\\nrespective Confidential Information be maintained in accordance with this Agreement. Therefore, the Parties agree as follows:\\n1. Definitions.\\n1.1\\n“Affiliate” means, with respect to a Party, any person, cor